#### Names of people in the group

Please write the names of the people in your group in the next cell.

Name of person A: Dominik Ucher

Name of person B

In [0]:
string1 = "<tag1>"
a = string1.split(">")
b = a[:-1]
print(b)
print(len(b))

['<tag1']
1


In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-84786e9e-ebbc-4b80-8efe-690a54d52d8c/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from collections import Counter
from pyspark.sql.functions import desc

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users.parquet")
posts_df = load_df("/user/hive/warehouse/posts.parquet")

#### Subtask 1: implementing two functions
Implement these two functions:
1. 'compute_pearsons_r' that receives a DataFrame and two column names and returns the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between values of two columns;
2. 'make_tag_graph' that in the input receives the DataFrame containing the records related to 'questions' and returns a DataFrame with two columns 'u' and 'v'; the record for row i from the resulting DataFrame is a tuple (u_i, v_i). u_i and v_j are distinct tags and have appeared together for a question.

Please note that you should implement the 'compute_pearsons_r' yourself, so you should not use the 'DataFrame.stat.corr' method. Nevertheless, you can use 'DataFrame.stat.corr' to verify the correctness of your implementation.

In [0]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.functions import avg, stddev_pop


spark = SparkSession.builder.appName("Task4").getOrCreate()

def compute_pearsons_r(df: DataFrame, col1: str, col2: str) -> float:
    df.createOrReplaceTempView("df")
    result = spark.sql(f"SELECT (AVG({col1} * {col2}) - (AVG({col1}) * AVG({col2}))) / (stddev_pop({col1}) * stddev_pop({col2})) pearsons_r FROM df")

    pearsons_r = result.collect()[0]['pearsons_r']

    return pearsons_r



def make_tag_graph(df: DataFrame) -> DataFrame:
    df.createOrReplaceTempView("df")
    data = []  # Helper dataset
    tagsCollect = df.collect()

    for row in tagsCollect:
        tags = row['Tags']
        
        tagdata = tags.split(">")
        tagdata = tagdata[:-1]
        if (len(tagdata) == 1):
            tag = tagdata[0].replace("<", "").strip()
            data.append((tag,tag))
        else:
            for i in range(len(tagdata) - 1):
                tagdata[i] = tagdata[i].replace("<", "")
                u = tagdata[i]
                for j in range(i+1, len(tagdata)):
                    v = tagdata[j].replace("<", "").strip()
                    data.append((u, v))
                    data.append((v, u))

    tag_graph = spark.createDataFrame(data, ["u", "v"])
    return tag_graph



#### Subtask 2: implementing three functions
Impelment these three functions:
1. 'get_nodes' that, given the result from execution of 'make_tag_graph', returns a DataFrame with one column named 'id' that includes the tags that have appeared in the tag graph;
2. 'get_edges' that, given the result from execution of 'make_tag_graph', returns a DataFrame with two columns 'src' and 'dst' where 'src' is the source node and 'dst' is the destination node.


Note that the term 'tag graph' in this context refers to the DataFrame reuturned by executing 'make_tag_graph'. Furthermore, 'src' and 'dst' are distinct, so 'src' != 'dst'.

In [0]:
def get_nodes(df: DataFrame) -> DataFrame:
    df.createOrReplaceTempView("df")

    tags = spark.sql("SELECT DISTINCT u as id FROM df WHERE u IS NOT NULL AND u != '' UNION SELECT DISTINCT v as id FROM df WHERE v IS NOT NULL AND v != ''")

    tag_graph = tags.select("id").distinct()

    return tag_graph


def get_edges(df: DataFrame) -> DataFrame:
    df.createOrReplaceTempView("df")
    data = []  
    tagsCollect = df.collect()
    
    for row in tagsCollect:
        u = row['u']
        v = row['v']

        if u != v:
            data.append((u, v))

    edge_graph = spark.createDataFrame(data, ["src", "dst"])

    return edge_graph

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask3(unittest.TestCase):
  
  error_threshold = 0.03
  
  def test_corr1(self):
    # Pearson correlation coefficient between 'user reputation' and 'upvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
    self.assertLessEqual(abs(result-0.5218138310114108), self.error_threshold)
    print(result)
  
  def test_corr2(self):
    # Pearson correlation coefficient between 'user reputation' and 'downvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "DownVotes")
    self.assertLessEqual(abs(result-0.1473558141546844), self.error_threshold)
    print(result)

  def test_corr3(self):
    # Pearson correlation coefficient between 'question score' and the 'number of answers' it received
    result = compute_pearsons_r(posts_df[posts_df["PostTypeId"] == 1], "Score", "AnswerCount")
    self.assertLessEqual(abs(result-0.47855272641249674), self.error_threshold)
    print(result)
    
  def test_make_tag_graph(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    self.assertIsInstance(result, DataFrame)
    
    coulmn_names = Counter(map(str.lower, ['u', 'v']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)), "Missing column(s) or column name mismatch")
    
    display(result)
    
    self.assertEqual(result.count(), 228830)
    
  def test_get_nodes(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    self.assertEqual(n.count(), 638)
    n.show()

  def test_get_edges(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    e = get_edges(result)
    
    coulmn_names = Counter(map(str.lower, ['src', 'dst']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, e.columns)), "Missing column(s) or column name mismatch")
    
    self.assertEqual(e.count(), 225290)
    e.show()
    

0.5218166861948921
0.14735661849993056
0.4753098634046649
+----------------+----------------+
|             src|             dst|
+----------------+----------------+
|       education|     open-source|
|     open-source|       education|
|     data-mining|     definitions|
|     definitions|     data-mining|
|machine-learning|         bigdata|
|         bigdata|machine-learning|
|machine-learning|          libsvm|
|          libsvm|machine-learning|
|         bigdata|          libsvm|
|          libsvm|         bigdata|
|         bigdata|     scalability|
|     scalability|         bigdata|
|         bigdata|      efficiency|
|      efficiency|         bigdata|
|         bigdata|     performance|
|     performance|         bigdata|
|     scalability|      efficiency|
|      efficiency|     scalability|
|     scalability|     performance|
|     performance|     scalability|
+----------------+----------------+
only showing top 20 rows

+--------------------+
|                  id|
+-----

u,v
machine-learning,machine-learning
education,open-source
open-source,education
data-mining,definitions
definitions,data-mining
databases,databases
machine-learning,bigdata
bigdata,machine-learning
machine-learning,libsvm
libsvm,machine-learning


Success

......
----------------------------------------------------------------------
Ran 6 tests in 41.509s

OK
Out[97]: <unittest.runner.TextTestResult run=6 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'User-Defined Functions (UDFs)', 'Data Locality', 'Bucketing', 'Distributed Filesystem' mean in the context of Spark?

Write your descriptions in the next cell.

### User-Defined Functions (UDFs)
UDFs are custom funcitons that users can define and apply to datasets and dataframes. The user can make UDFs in Python, Scala and Java. But these UDFs (Functions) can only be used on a single row/colum, so if you expect to use this function on the whole dataframe, then you must iterate through each row/colum to do so
### Data Locality
Spark is a data parallel processing framework. Therefore Data Locality means that it located the data so that it will execute tasks as close to where the data lives as possible, which in return minimizes data transfers. This makes is very efficient for handling big data. 
### Bucketing
Bucketing is a technique that is used to distribute large data across multiple "Buckets". Bucketing works during the creation of the DataFrame. It applies a Hash function to a column and divides the data in that column into buckets that match the given hash value. This reduces the amount of data that it needed to scan for query processing and results in faster executions and better resource utilization
### Distributed Filesystem
HDFS is an example for a distributed filesystem. They are designed to store large files across multiple physical machines. Spark uses it as a tool for running distributed computations over large datasets. This allows parallel processing of large datasets, something that is a key factor in Spark and processing large amounts of data. 